In [1]:
import requests
base_url = 'https://m.aliexpress.com/ajaxapi/EvaluationSearchAjax.do?type=all&pageSize=80&country=US'
def importReviews(doc_complete, product_id):
    totalPage = 10000000000000000000
    n = 1
    while (n<=totalPage):
        #print(n)
        url = base_url + "&productId="+str(product_id)+"&index="+str(n)
        #print(url)
        resp = requests.get(url)
        if resp.status_code != 200:
            # This means something went wrong.
            raise ApiError('GET /EvaluationSearchAjax/ {}'.format(resp.status_code))
        resp_json = resp.json()
        doc_complete_n = [review["buyerTranslationFeedback"] 
                          for review in resp_json["evaViewList"] 
                          if "buyerTranslationFeedback" in review.keys() 
                          #and float(review["buyerEval"])<50
                         ]
        doc_complete= doc_complete + doc_complete_n
        if (n==1):
            #print(resp_json)
            totalPage = int(resp_json["totalPage"])
            print(str(product_id)+" totalPage "+str(totalPage))
        print(".", end='')    
        n=n+1
    return doc_complete

In [2]:
#Xiaomi Redmi 4 Pro Prime
#  32752699189 Doogee x5 max 
# 32757712697 lenovo a5600 
# 32798494733 Lenovo ZUK Z2 
# 32791453530 ONEPLUS 3 T A3003 
# 32671947390  Blackview BV6000 
# 32719969124 Xiaomi Mi Max Mimax Pro 
# 32799364813 Oukitel U20 Plus Android 6.0
# 32611785137 Xiaomi Mi5 M5 Mi 5 Prime
# 32763320059
product_ids = [
32796300206,
32797368004,
32660593076,
32763320059,
32752699189,
32757712697,
32798494733,
32791453530,
32671947390,
32719969124,
32799364813,
32611785137,
32763320059]
doc_complete = []
for product_id in product_ids:
    doc_complete=importReviews(doc_complete,product_id)
print("count of reviews: "+ str(len(doc_complete)))



32796300206 totalPage 97
.................................................................................................32797368004 totalPage 18
..................32660593076 totalPage 11
...........32763320059 totalPage 107
...........................................................................................................32752699189 totalPage 12
............32757712697 totalPage 7
.......32798494733 totalPage 12
............32791453530 totalPage 9
.........32671947390 totalPage 7
.......32719969124 totalPage 8
........32799364813 totalPage 7
.......32611785137 totalPage 52
....................................................32763320059 totalPage 107
...........................................................................................................count of reviews: 4662


In [3]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    tokens = word_tokenize(doc.lower())
    normalized = " ".join(lemma.lemmatize(token) for token in tokens if token not in stop and token not in exclude)
    return normalized

doc_clean = [clean(doc) for doc in doc_complete]   
print(doc_clean)

['phone came fully discharged otherwise good satisfied purchase', 'phone reached mail russia lost way moscow volgograd seller returned money full refund ordered exactly phone came 2 week seller really go contact', 'recommend seller', 'excellent seller', '3 mo hope figment arrive rise disputa returned money still perform management lost lot time purchase first time happens chile discouraged ...', 'recommend', 'received two week phone excellent', 'phone fire nothing add took standard version 2 gig bought girlfriend mother instead old samsung packed well shipping 2 week st. petersburg vparivayut 11к communication money two think almost nothing daily task missing phone enclosed ruble slow decent camera review youtube recommend seller phone', 'good msk region reached 2 week', "excellent moved iphone regret easy use 'm glad", 'great phone thank', 'phone working standard hope also', 'fast shipping smart phone recommend', 'great phone second голдвея height recommend. thank seller fast delivery

In [4]:
import nltk

full_text = " ".join([review for review in doc_clean])

tokens = nltk.word_tokenize(full_text)

tagged = nltk.pos_tag(tokens)
print(tagged)

[('phone', 'NN'), ('came', 'VBD'), ('fully', 'RB'), ('discharged', 'VBN'), ('otherwise', 'RB'), ('good', 'JJ'), ('satisfied', 'JJ'), ('purchase', 'NN'), ('phone', 'NN'), ('reached', 'VBD'), ('mail', 'NN'), ('russia', 'NN'), ('lost', 'VBD'), ('way', 'NN'), ('moscow', 'NN'), ('volgograd', 'VBP'), ('seller', 'NN'), ('returned', 'VBD'), ('money', 'NN'), ('full', 'JJ'), ('refund', 'NN'), ('ordered', 'VBD'), ('exactly', 'RB'), ('phone', 'NN'), ('came', 'VBD'), ('2', 'CD'), ('week', 'NN'), ('seller', 'NN'), ('really', 'RB'), ('go', 'VB'), ('contact', 'JJ'), ('recommend', 'VB'), ('seller', 'NN'), ('excellent', 'JJ'), ('seller', 'NN'), ('3', 'CD'), ('mo', 'NN'), ('hope', 'NN'), ('figment', 'NN'), ('arrive', 'JJ'), ('rise', 'NN'), ('disputa', 'NN'), ('returned', 'VBD'), ('money', 'NN'), ('still', 'RB'), ('perform', 'VB'), ('management', 'NN'), ('lost', 'VBN'), ('lot', 'NN'), ('time', 'NN'), ('purchase', 'NN'), ('first', 'JJ'), ('time', 'NN'), ('happens', 'VBZ'), ('chile', 'JJ'), ('discouraged', 

In [5]:
#nouns = [token for token, pos in tagged if pos.startswith('N')]
#print(nouns)
  	
grammar = r"""
  NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
"""
cp = nltk.RegexpParser(grammar)
noun_phrase_tagged = cp.parse(tagged)
print(noun_phrase_tagged)
#doc_complete = nouns

(S
  (NOUN_PHRASE phone/NN came/VBD fully/RB)
  discharged/VBN
  (NOUN_PHRASE
    otherwise/RB
    good/JJ
    satisfied/JJ
    purchase/NN
    phone/NN
    reached/VBD
    mail/NN
    russia/NN)
  lost/VBD
  (NOUN_PHRASE way/NN moscow/NN volgograd/VBP seller/NN)
  returned/VBD
  (NOUN_PHRASE money/NN full/JJ refund/NN)
  ordered/VBD
  (NOUN_PHRASE exactly/RB phone/NN came/VBD 2/CD week/NN seller/NN)
  really/RB
  go/VB
  contact/JJ
  recommend/VB
  (NOUN_PHRASE seller/NN excellent/JJ seller/NN)
  (NOUN_PHRASE
    3/CD
    mo/NN
    hope/NN
    figment/NN
    arrive/JJ
    rise/NN
    disputa/NN)
  returned/VBD
  (NOUN_PHRASE money/NN still/RB)
  perform/VB
  (NOUN_PHRASE management/NN lost/VBN lot/NN time/NN purchase/NN)
  (NOUN_PHRASE first/JJ time/NN happens/VBZ chile/JJ)
  discouraged/VBD
  .../:
  recommend/VB
  received/VBD
  (NOUN_PHRASE
    two/CD
    week/NN
    phone/NN
    excellent/JJ
    phone/NN
    fire/NN
    nothing/NN)
  add/VB
  took/VBD
  (NOUN_PHRASE standard/JJ ve

In [6]:
def traverse(t,noun_phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            [noun_phrases.append(word) for (word,pos) in t.leaves() if pos.startswith("N")]
            #print(statement)
            #noun_phrases = noun_phrases.append(noun_phrase)
        # Now we know that t.node is defined
        #print('(', t.label(), end=" ")
        for child in t:
            traverse(child,noun_phrases)
        #print(')', end=" ")
    return noun_phrases

noun_phrases=[]
nouns=traverse(noun_phrase_tagged,noun_phrases)
print(nouns)
doc_complete = nouns

['phone', 'purchase', 'phone', 'mail', 'russia', 'way', 'moscow', 'seller', 'money', 'refund', 'phone', 'week', 'seller', 'seller', 'seller', 'mo', 'hope', 'figment', 'rise', 'disputa', 'money', 'management', 'lot', 'time', 'purchase', 'time', 'week', 'phone', 'phone', 'fire', 'nothing', 'version', 'gig', 'mother', 'samsung', 'week', 'st.', 'petersburg', 'vparivayut', 'communication', 'money', 'nothing', 'task', 'phone', 'decent', 'camera', 'review', 'seller', 'phone', 'msk', 'region', 'week', 'use', 'phone', 'thank', 'phone', 'hope', 'phone', 'recommend', 'phone', 'голдвея', 'recommend', 'seller', 'delivery', 'day', 'region', 'phone', 'голдвея', 'recommend', 'seller', 'delivery', 'day', 'region', 'excellent', 'purchase', 'band', 'work', 'phone', 'payment', 'recommend', 'case', 'material', 'delicate', 'smartphone', 'seller', 'russia', 'number', 'day', 'dispute', 'aliekspress', 'money', 'problem', 'day', 'time', 'phone', 'xiaomi', 'seller', 'complaint', 'smartphone', 'post', 'custom', '

In [7]:
from nltk import FreqDist
import nltk
#print(doc_clean)
#words = nltk.tokenize.word_tokenize(doc_clean)
all_reviews_in_one_string = " ".join([doc for doc in doc_clean])

words = nltk.tokenize.word_tokenize(all_reviews_in_one_string)
fdist = FreqDist(noun_phrases) 
frequent_words_freq = fdist.most_common(200)
print(frequent_words_freq )
#fdist.plot()
frequent_words= [w for (w,c) in frequent_words_freq]
#print(frequent_words)



[('phone', 3357), ('seller', 2354), ('day', 1623), ('everything', 1093), ('week', 879), ('delivery', 782), ('order', 736), ('work', 714), ('film', 527), ('time', 508), ('thank', 412), ('gift', 392), ('money', 367), ('recommend', 328), ('quality', 295), ('glass', 266), ('description', 254), ('case', 240), ('problem', 236), ('firmware', 231), ('region', 228), ('super', 215), ('adapter', 214), ('packaging', 201), ('purchase', 197), ('track', 191), ('screen', 190), ('mail', 187), ('month', 187), ('product', 187), ('camera', 162), ('custom', 160), ('box', 154), ('price', 151), ('parcel', 150), ('\\\\_', 139), ('battery', 134), ('advise', 126), ('russia', 125), ('thanks', 125), ('complaint', 120), ('use', 118), ('silicone', 116), ('shipping', 116), ('store', 113), ('smartphone', 112), ('package', 104), ('post', 99), ('fine', 98), ('air', 93), ('version', 91), ('cover', 90), ('star', 89), ('dispute', 88), ('charge', 88), ('way', 83), ('outlet', 78), ('extension', 77), ('device', 77), ('questi

def clean(doc):
    tokens = word_tokenize(doc)
    only_frqt_words = " ".join([token for token in tokens if token in frequent_words])
    return only_frqt_words 

#doc_clean2 = [clean(doc) for doc in doc_clean] 
doc_clean2 = [clean(doc).split() for doc in doc_clean] 
#print(doc_clean2)

Now find snippets in the review with these frequent nouns

In [9]:
def traverse(t,noun,phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            phrase = " ".join([word for (word,pos) in t.leaves()])
            if noun in phrase.split():
                phrases.append(phrase)
                #print(phrase)
        for child in t:
            traverse(child,noun,phrases)
    return phrases

from nltk.sentiment.vader import SentimentIntensityAnalyzer

dict_noun_abs_sentiment = {}
for noun in frequent_words:
    phrases=[]
    phrases=traverse(noun_phrase_tagged,noun,phrases)
    print(phrases[1:3])
    #doc_complete = nouns

    sid =  SentimentIntensityAnalyzer()
    sentiments = sum(abs(sid.polarity_scores(phrase)["compound"]) for phrase in phrases)/len(phrases)
    print(noun + " has absolute avg sentiment " + str(sentiments))
    dict_noun_abs_sentiment[noun]=sentiments

['otherwise good satisfied purchase phone reached mail russia', 'exactly phone came 2 week seller']
phone has absolute avg sentiment 0.4011995170637475
['exactly phone came 2 week seller', 'seller excellent seller']
seller has absolute avg sentiment 0.41141470085470305
['18 day zabaikalsk region excellent', '60 day come opened dispute aliekspress']
day has absolute avg sentiment 0.2350794608472406
['everything tracked day packaging quality', 'excellent everything super order']
everything has absolute avg sentiment 0.389847232824427
['two week phone excellent phone fire nothing', '2 week st. petersburg vparivayut 11к communication money']
week has absolute avg sentiment 0.27101458094144676
['seller fast delivery', 'dostavuka quicker delivery two week date payment honest perfect work']
delivery has absolute avg sentiment 0.350065155440415
['order quickly definitely still', 'order quickly definitely still']
order has absolute avg sentiment 0.30924188626907084
['dostavuka quicker delivery 

['extension megabonus save 2.35 extension megabonus help', 'product purchased extension megabonus']
extension has absolute avg sentiment 0.5687836363636366
['excellent device always', 'fine money excellent device phone']
device has absolute avg sentiment 0.4116719512195122
['content question seller explained everything response', 'normal question answered course заигнорил custom control']
question has absolute avg sentiment 0.26774999999999993
['two week phone excellent phone fire nothing', '2 week excellent']
excellent has absolute avg sentiment 0.6876212121212134
['fast two week lipetsk region mail singapore track', '29 phone элисте singapore believe']
singapore has absolute avg sentiment 0.15440495049504946
['review super fast delivery yekaterinburg 2 week phone price', 'write review']
review has absolute avg sentiment 0.3116150684931507
['dostavuka quicker delivery two week date payment honest perfect work', 'within 12 hour payment ordered seller second phone']
payment has absolute

['two month waiting end protection order', '17 day gift put glass protection screen scratch popolzuyutsya']
protection has absolute avg sentiment 0.26720000000000005
['many year use smartphones already', 'second phone year everything perfect came 18 day zaporozhye']
year has absolute avg sentiment 0.18363125
['good speaker camera included adapter socket', 'good film adapter socket podarok always голдвея']
socket has absolute avg sentiment 0.4835999999999999
['fine well fast delivery le two week mo', 'tape came le 2 week krasnoyarsk']
le has absolute avg sentiment 0.276876
['third phone store well always great quality seller check phone', 'check link // letyshops']
check has absolute avg sentiment 0.36589999999999995
['son dovolen came ulyanovsk two week phone', 'protective film budu ordered another smart son phone']
son has absolute avg sentiment 0.332285294117647
['thank seller good reached st. petersburg 12 day description coincides', 'petersburg husband happy thank seller']
petersbu

['see possible', 'global version see whether problem']
see has absolute avg sentiment 0.25188157894736846
['one star seller communicate request send phone', 'another order store thank thank seller communication seemed order request seller']
request has absolute avg sentiment 0.34715000000000007
['quickly camera seems 13мегапикселей well super body phone fire', 'several place scratch body negative perhaps great body everything work']
body has absolute avg sentiment 0.26723076923076927
['video video reason sent track number', 'video video reason sent track number']
video has absolute avg sentiment 0.24664074074074077
['great phone camera 5 smart many function', 'firmware function work 100']
function has absolute avg sentiment 0.34565454545454555
['general regret ordered let', 'old alcatel idol mini regret buying well thank quickly']
regret has absolute avg sentiment 0.38977560975609754
['excellent let', 'let checked prodovets delivery']
let has absolute avg sentiment 0.2177375
['complete

In [10]:
print( [ (key,'{:01.2f}'.format(val)) for (key,val) in dict_noun_abs_sentiment.items() if val >0.1])

[('phone', '0.40'), ('seller', '0.41'), ('day', '0.24'), ('everything', '0.39'), ('week', '0.27'), ('delivery', '0.35'), ('order', '0.31'), ('work', '0.36'), ('film', '0.40'), ('time', '0.23'), ('thank', '0.59'), ('gift', '0.58'), ('money', '0.33'), ('recommend', '0.58'), ('quality', '0.42'), ('glass', '0.27'), ('description', '0.36'), ('case', '0.35'), ('problem', '0.37'), ('firmware', '0.31'), ('region', '0.22'), ('super', '0.71'), ('adapter', '0.38'), ('packaging', '0.44'), ('purchase', '0.40'), ('track', '0.25'), ('screen', '0.31'), ('mail', '0.18'), ('month', '0.25'), ('product', '0.40'), ('camera', '0.41'), ('custom', '0.23'), ('box', '0.24'), ('price', '0.48'), ('parcel', '0.21'), ('\\\\_', '0.19'), ('battery', '0.34'), ('advise', '0.38'), ('russia', '0.24'), ('thanks', '0.59'), ('complaint', '0.32'), ('use', '0.26'), ('silicone', '0.38'), ('shipping', '0.33'), ('store', '0.44'), ('smartphone', '0.36'), ('package', '0.22'), ('post', '0.12'), ('fine', '0.42'), ('air', '0.25'), ('

In [11]:
import pymongo
client = pymongo.MongoClient()
mydb = client['y2buy_1']
my_collection = mydb['features']
result = mydb.features.create_index([('feature', pymongo.ASCENDING)],unique=True)

In [12]:
features=  [ {"feature" : key} for (key,val) in dict_noun_abs_sentiment.items() if val >0.1]

In [13]:
my_collection.insert_many(features)